In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
#call library

import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

from preprocesslib import preprocessEach, makeY

In [12]:
xGrowths = preprocessEach('growth')
xValues = preprocessEach('value')
xOverlaps = preprocessEach('overlap')

In [16]:
#bootstrap -이거 재현언니 더 좋은 코드 있다 하지 않았나요....ㅠ
def bootstrap(data, n_samples):
    # generate bootstrap sample
    samples = []
    # sampling n_samples times
    for i in range(n_samples):
        # randomly sample from data with replacement
        sample = data[np.random.choice(len(data), size=1, replace=True)]
        # append sample to ndarray
        samples = np.append(samples, sample)
    return samples

n_samples = len(xValues) - len(xGrowths)
xGrowths_bootstrapped = bootstrap(xGrowths, n_samples)
xGrowths = np.concatenate((xGrowths, xGrowths_bootstrapped), axis=0)

n_samples = len(xValues) - len(xOverlaps)
xOverlaps_bootstrapped = bootstrap(xOverlaps, n_samples)
xOverlaps = np.concatenate((xOverlaps, xOverlaps_bootstrapped), axis=0)

In [21]:
#bootstrap
n_samples = len(xValues)
xGrowths = xGrowths[np.random.choice(len(xGrowths), size=n_samples, replace=True)]
xOverlaps = xOverlaps[np.random.choice(len(xOverlaps), size=n_samples, replace=True)]

In [36]:
#make y values
yGrowths = makeY('growth', n_samples)
yValues = makeY('value', n_samples)
yOverlaps = makeY('overlap', n_samples)

In [24]:
x = np.concatenate((xGrowths, xValues, xOverlaps), axis=0)
y = np.concatenate((yGrowths, yValues, yOverlaps), axis=0)

In [33]:
#minmax scaling
scaler = MinMaxScaler()
for i in range(len(x)):
    x[i] = scaler.fit_transform(x[i])

In [38]:
xGrowths[0]

array([[ 3.58905914, 12.63720875,  0.64185268,  1.91397692,  0.        ,
         3.00991691],
       [ 3.58905914, 12.47991311,  0.69314718,  1.89911803,  0.        ,
         2.79902198],
       [ 3.56246553, 12.60686351,  0.74193662,  2.01223273,  0.        ,
         2.53142667],
       ...,
       [ 3.77597415, 13.12216536,  0.76080554,  1.77495261,  3.77986208,
         0.        ],
       [ 3.78168649, 12.7346429 ,  0.69813389,  1.93585986,  3.77881154,
         0.        ],
       [ 3.76166513, 12.64690811,  0.83724647,  1.97685459,  3.77368004,
         0.        ]])

In [34]:
x

array([[[0.20132041, 0.32905387, 0.12831375, 0.72446692, 0.        ,
         0.67447325],
        [0.20132041, 0.29829786, 0.15466084, 0.71884262, 0.        ,
         0.62721514],
        [0.17558879, 0.32312047, 0.17972122, 0.76165811, 0.        ,
         0.5672514 ],
        ...,
        [0.38217688, 0.42387742, 0.18941312, 0.67184428, 0.86225378,
         0.        ],
        [0.38770407, 0.34810517, 0.15722223, 0.73274991, 0.86201413,
         0.        ],
        [0.36833167, 0.33095039, 0.22867651, 0.74826699, 0.86084354,
         0.        ]],

       [[0.00644864, 0.37567181, 0.13821767, 0.70852338, 0.        ,
         0.63855115],
        [0.00188826, 0.27522688, 0.12102619, 0.7240908 , 0.        ,
         0.64904854],
        [0.00229207, 0.37496392, 0.13172283, 0.71441818, 0.        ,
         0.65907219],
        ...,
        [0.90982473, 0.30722621, 0.47563833, 0.74951214, 0.9764121 ,
         0.76592186],
        [0.92439052, 0.30335646, 0.42545492, 0.68159994, 0.977

In [26]:
def splitData(x, y, train_ratio=0.8):
    # shuffle data
    permutation = np.random.permutation(len(x))
    x = x[permutation]
    y = y[permutation]

    # split data
    n_train_samples = int(train_ratio * len(x))
    x_train, x_test = x[:n_train_samples], x[n_train_samples:]
    y_train, y_test = y[:n_train_samples], y[n_train_samples:]

    return x_train, x_test, y_train, y_test

In [27]:
x_train, x_test, y_train, y_test = splitData(x, y)

In [29]:
#modify NaN to 0
x_train = np.nan_to_num(x_train, nan=0)
x_test = np.nan_to_num(x_test, nan=0)